In [1]:
import discogs_client

In [2]:
def readUserToken():
    try:
        with open("user_token.env", "r") as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return {}
        
d = discogs_client.Client('dardok_app/0.1', user_token='{}'.format(readUserToken().rstrip()))

In [3]:
label_ids = "251117,113"
min_average = 4.8

In [4]:
# COMMON FUNCTIONS

def getLabelsIdsList(label_ids):
    """ A function to convert elements to integers """
    label_id_list = [int(label_id) for label_id in label_ids.split(',')]
    return label_id_list

def getFilteredReleaseListFromLabel(label):
    """ A function that returns all the filtered releases from label """
    filtered_release_list = []
    print()
    print("Processing label: {} {}...".format(label.name, label.url))
    for k in range(1, result_label.releases.pages+1):
        print()
        print("Page: {}/{}".format(k,result_label.releases.pages))
        print(result_label.releases.page(k))
        for item in result_label.releases.page(k):
            release = d.release(item.id)       
            if release.community.rating.average >= min_average and release.formats[0]['name']=='Vinyl' and 'Electronic' in release.genres:
                filtered_release_list.append(release)
    return filtered_release_list

def funcYear(e):
    """ A function that returns the 'year' value """
    return e.year

def remove_duplicates(input_list):
    """ A function to remove duplicates from a release input_list by id """
    unique_list = []
    seen_ids = set()

    for item in input_list:
        item_id = item.id
        if item_id not in seen_ids:
            seen_ids.add(item_id)
            unique_list.append(item)

    return unique_list

def get_artists(release):
    """ A function to return an artists list from release """
    artists = []
    for artist in release.artists:
        artists.append(artist.name)
    return artists

def get_labels(release):
    """ A function to return an labels list from release """
    labels = []
    for label in release.labels:
        labels.append(label.name)
    return labels

def writeOutputHtml(label, release_list):
    """ A function to write output as HTML """

    htmlString = "<html>\n<head>\n<title>Filtered Releases for label {} - id: {}</title>\n</head>\n \
                    <body>\n \
                        <h1>Filtered Releases for label {} - id: {}</h1>\n \
                        <h2>Url: <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a><h2>\n \
                        <br> \
                        Minimum community rating average filter: <span style='color: #ff0000'>{}</span> \n \
                        <br> \
                        Number of results: {} \
                    ".format(label.name, label.id, label.name, label.id, label.url, label.url, min_average, len(release_list))

    htmlString += "<ul style='list-style: none;'>\n"
    k = 0
    for release in release_list:
        htmlString += "<br>\n"
        k += 1
        htmlString += "<span style='color: #5f5f5f'>Result {}/{}</span>\n".format(k, len(release_list)) 
        htmlString += "<li><span style='color: #9b9b9b'>catno:</span> {}</li>\n".format(release.data['labels'][0]['catno'])
        htmlString += "<li><span style='color: #9b9b9b'>Title:</span> {}</li>\n".format(release.title)
        artists = get_artists(release)
        htmlString += "<li><span style='color: #9b9b9b'>Artists:</span> {}</li>\n".format(artists)
        htmlString += "<li><span style='color: #9b9b9b'>Year:</span> {}</li>\n".format(release.year)
        labels = get_labels(release)
        htmlString += "<li><span style='color: #9b9b9b'>Labels:</span> {}</li>\n".format(labels)
        htmlString += "<li><span style='color: #9b9b9b'>Genres:</span> {}</li>\n".format(release.genres)
        htmlString += "<li><span style='color: #9b9b9b'>Styles:</span> {}</li>\n".format(release.styles)
        htmlString += "<li><span style='color: #9b9b9b'>Formats:</span> {}</li>\n".format(release.formats)
        htmlString += "<li><span style='color: #9b9b9b'>Community rating average/count:</span> <span style='color: #ff0000'>{}</span>/<span style='color: #ff8000'>{}</span></li>\n".format(release.community.rating.average, release.community.rating.count)
        htmlString += "<li><span style='color: #9b9b9b'>Url:</span> <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a></li>\n".format(release.url, release.url)

    htmlString += "</ul>\n"
    
    htmlString += "</body>\n \
                  </html>"

    file_name = "{}.html".format(label.url.lstrip('https://www.discogs.com/label/'))
    
    with open("./outputHtml/{}".format(file_name), "w") as file:
        file.write(htmlString)

    print()
    print("Written HTML output file to ./outputHtml/{}".format(file_name))

In [ ]:
labels_id_list = getLabelsIdsList(label_ids)
for label_id in labels_id_list:
    result_label = d.label(label_id)
    filtered_release_list = getFilteredReleaseListFromLabel(result_label)
    filtered_release_list.sort(reverse=True, key=funcYear)
    unique_filtered_release_list = remove_duplicates(filtered_release_list)
    writeOutputHtml(result_label, unique_filtered_release_list)
print()
print("Process finished")


Processing label: Livity Sound https://www.discogs.com/label/251117-Livity-Sound...

Page: 1/3
[<Release 5530070 'A Made Up Sound Remixes 1 & 2'>, <Release 5626051 'Ghost-202 & Beneath Remixes'>, <Release 5693713 'Surgeon & Nick Höppner Remixes'>, <Release 5763421 'MM/KM & Dresvn Remixes'>, <Release 21222433 'Tessela & Stenny & Andrea Remixes'>, <Release 6188944 'Tessela & Stenny & Andrea Remixes'>, <Release 5460506 'MMM & Pangaea Remixes'>, <Release 3433399 'Beneath Radar'>, <Release 2939256 'Beneath Radar'>, <Release 3433422 'Sister'>, <Release 3127874 'Sister'>, <Release 3399013 '#3 PEV'>, <Release 3533546 'More Games / Jam01'>, <Release 4663619 'Velez / Rendering'>, <Release 4901648 'End Point / Vapours'>, <Release 5013363 'Surge / Remnants'>, <Release 5460506 'MMM & Pangaea Remixes'>, <Release 6208258 'MMM & Pangaea Remixes'>, <Release 5530070 'A Made Up Sound Remixes 1 & 2'>, <Release 5626051 'Ghost-202 & Beneath Remixes'>, <Release 5693713 'Surgeon & Nick Höppner Remixes'>, <Re